### Cloud VPC Peering<br>

***

<br>

Normally VPCs would travel through public routing to communicate with each other<br>
but Cloud offers VPC peering<br>

**VPC Peering**:<br>

- Private connectivity across two VPC networks(RFC 1918)<br>
- Peer across the same or different projects and organizations<br>
- Reduces network latency<br>
- Increases network security<br>
- Reduces network costs<br>

They're individually administered<br>
So when you set up VPC, you have to go into each one and setup<br>

both side has to match in order to be active<br>
one side can delete peering association at any given time<br>

VPC peers always exchange all subnet route<br>
you also have an option of exchanging custom routes<br>

subnet and static routes are global<br>
dynamic routes can be regional or global<br>

given VPC network can peer with multiple VPC networks<br>
<br>

**Restrictions in VPC peering:**<br>

CIDR range CANNOT overlap with a static route with another peer<br>
this rule covers both subnet routes and static routes<br>
Cloud performs a check on new subnet range<br>
If it finds anything, the creation or expansion will fail<br>

VPC network peering doesn't provide granular route controls to filter out<br>
which subnet CIDR ranges are reachable across peered networks<br>
these are handled by the Firewall rules<br>

To allow ingress traffic from VM instances in a peer network, you must create<br>
inagress allow firewall rules. By default, ingress traffic to VMs are blocked<br>

Transitive peering is NOT supported<br>
network-a -> network-b -> network-c<br>
so network a cannot commuicate with network c<br>
unless network a is directly peering with network c<br>

Internal DNS is not accessible for compute engine in peered network<br>
As they need an IP to communicate.<br>

**Demo**<br>

We'll establish a peering connection between 2 VPCs in 2 separate projects<br>
To verify, we'll create 2 instances in each network and ping each other<br>
This is similar to Custom VPC demo, but adding another layer<br>
<br>

**Creating VPC networks for each project:**<br>

In Project 1<br>
Google Cloud -> VPC Network -> create new VPC<br>
name: network-a<br>
subnet: custom, subnet-a<br>
region: us-east1<br>
IP: 10.0.0.0/20<br>
<br>

Move to Project 2<br>
Create VPC network here also<br>
name: network-b<br>
subnet: custom, subnet-b<br>
region: us-east4<br>
IP: 10.4.0.0/20<br>
<br>

**Creating Firewall rules for each project:**<br>

Back to Project 1<br>
VPC Network -> Firewall Rules -> Create a new Firewall<br>
name: firewall-a<br>
network: network-a(we create earlier)<br>
priority: 1000<br>
direction of traffic: ingress<br>
action on match: allow<br>
target: all instances in the network<br>
source filter: IP ranges<br>
source IP: 0.0.0.0/0, in real environment, you should only communicate with the IP you're communicating with<br>
protocols and ports: TCP: 22, other protocols: icmp<br>
<br>

Move to Project 2<br>
name: firewall-b<br>
network: network-b<br>
priority: 1000<br>
direction of traffic: ingress<br>
action on match: allow<br>
target: all instances in the network<br>
source filter: IP ranges<br>
source IP: 0.0.0.0/0<br>
protocols and ports: TCP: 22, other protocols: icmp<br>
<br>

**Creating instances for each project:**<br>

Still in Project 2<br>
Google Cloud -> Compute Engine -> create my instance<br>
name: instance-b<br>
region: us-east4, us-east4-c<br>
machine-type: e2-micro<br>
networking: network interface: network-b<br>
<br>

Move to Project 1<br>
Google Cloud -> Compute Engine -> create my instance<br>
name: instance-a<br>
region: us-east1, us-east1-c<br>
machine-type: e2-micro<br>
networking: network interface: network-a<br>
<br>

**How to set up a VPC peering connection:**<br>

Still in Project 1<br>
Google Cloud -> VPC Network -> VPC Networking Peering -> Create connection<br>
name: peering-ab<br>
vpc network: network-a<br>
peered VPC > another project: projectID: project-b, VPC network: network-b<br>
<br>

Status is marked inactive, because other side haven't activated theirs yet<br>

Move to Project 2<br>
name: peering-ba<br>
vpc network: network-b<br>
peered VPC > another project: projectID: project-a, VPC network: network-a<br>
<br>

**To verify it's working:**<br>

Still in Project 2<br>
Google Cloud -> Compute Engine -> VM Instances -> SSH into instance-b<br>
> ping 10.0.0.2(this is instance-a Internal IP)<br>
> Ctrl + C to stop the ping<br>

Delete everything to end the demo<br>
<br>